<a href="https://colab.research.google.com/github/christopherjneelankavil/bigram/blob/main/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy matplotlib pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp311-cp311-linux_x86_64.whl size=226201 sha256=3d5f7bbbc90b11e8c417b4714be3b135eeaaa127a1db6b249fe75b2d289e5b7a
  Stored in directory: /root/.cache/pip/wheels/5e/a6/15/41db3b28c85c319634deaac952ffb8869d199d50d71332c119
Successfully built pylzma


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
block_size = 8
batch_size = 4

In [ ]:
with open("/content/wizard_of_oz.txt","r",encoding="utf-8") as f:
  text = f.read()

chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)
print(len(chars))

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
69


In [ ]:
string_to_int = {ch : i for i,ch in enumerate(chars)}
int_to_string = {i : ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([68,  1,  1, 30, 44, 41,  1, 33, 51, 50, 40, 41, 54, 42, 57, 48,  1, 33,
        45, 62, 37, 54, 40,  1, 51, 42,  1, 25, 62,  0,  0,  0,  0,  0, 13, 44,
        37, 52, 56, 41, 54,  1, 19,  0, 30, 44, 41,  1, 13, 61, 39, 48, 51, 50,
        41,  0,  0,  0, 14, 51, 54, 51, 56, 44, 61,  1, 48, 45, 58, 41, 40,  1,
        45, 50,  1, 56, 44, 41,  1, 49, 45, 40, 55, 56,  1, 51, 42,  1, 56, 44,
        41,  1, 43, 54, 41, 37, 56,  1, 21, 37, 50, 55, 37, 55,  1, 52, 54, 37,
        45, 54, 45, 41, 55,  5,  1, 59, 45, 56, 44,  1, 31, 50, 39, 48, 41,  0,
        18, 41, 50, 54, 61,  5,  1, 59, 44, 51,  1, 59, 37, 55,  1, 37,  1, 42,
        37, 54, 49, 41, 54,  5,  1, 37, 50, 40,  1, 11, 57, 50, 56,  1, 15, 49,
         5,  1, 59, 44, 51,  1, 59, 37, 55,  1, 56, 44, 41,  1, 42, 37, 54, 49,
        41, 54, 65, 55,  1, 59, 45, 42, 41,  7,  1, 30, 44, 41, 45, 54,  0, 44,
        51, 57])


In [ ]:
## Train and val split

n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(ix)

  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([ 30600,  73454, 107397, 137031])
inputs:
tensor([[48, 56,  1, 58, 41, 54, 61,  1],
        [37, 47,  1, 51, 42,  1, 45, 56],
        [ 0, 66, 19,  1, 59, 45, 48, 48],
        [56,  5,  1, 59, 45, 56, 44,  1]])
targets:
tensor([[56,  1, 58, 41, 54, 61,  1, 52],
        [47,  1, 51, 42,  1, 45, 56,  5],
        [66, 19,  1, 59, 45, 48, 48,  1],
        [ 5,  1, 59, 45, 56, 44,  1, 37]])


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([68]) target is tensor(1)
when input is tensor([68,  1]) target is tensor(1)
when input is tensor([68,  1,  1]) target is tensor(30)
when input is tensor([68,  1,  1, 30]) target is tensor(44)
when input is tensor([68,  1,  1, 30, 44]) target is tensor(41)
when input is tensor([68,  1,  1, 30, 44, 41]) target is tensor(1)
when input is tensor([68,  1,  1, 30, 44, 41,  1]) target is tensor(33)
when input is tensor([68,  1,  1, 30, 44, 41,  1, 33]) target is tensor(51)


In [ ]:
from math import log
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None

    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
      # index is (B, T) array of indices in the current context
      for _ in range(max_new_tokens):
          # get the predictions
          logits, loss = self.forward(index) # Pass targets=None to forward
          # focus only on the last time step
          logits = logits[:, -1, :] # becomes (B, C)
          # apply softmax to get probabilities
          probs = F.softmax(logits, dim=-1) # (B, C)
          # sample from the distribution
          index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
          # append sampled index to the running sequence
          index = torch.cat((index, index_next), dim=1) # (B, T+1)
      return index

model = BigramLanguageModel(vocab_size=vocabulary_size)
context = torch.zeros((1,1), dtype = torch.long)
generated_chars = decode(model.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


RQ﻿D‘-fC— rwX“Typi,Es—”aaAaDZ—O,NEUYTczgfr-!:Ouj ?.pnh“Ay!B‘﻿wclxDA“WV:o-eMz”KRxyiR,F‘OxWqnrss!MqnE﻿iAqEuz’QiMm)LAzL?o!LXQLbO;pbB—TXPFwDAiKoiRbJR-XyjSZuz‘BeXzfYkDUPF.pVw,saSOZJDAD‘tU
Il’:XM?O“q!saDU?oH
? JOH‘bnb;JJytRJ Fb;WxN?o-GQWKnjRV﻿﻿FyS’KoiCpuFeyi”KUr:a—”.S (-TBe-aWqjkqHZJ:Pld—RwQL,﻿rQB;-XbpnbZ—TFsEh,d —xAH-f-zYYr?BEkypHIqHIoHIE﻿rQJ !XhAw)Ei”t—:okC—ExH!,QZ!HI‘By-,—’J ;UoRbZhiRPYkDAO(TwIHiz’tiDFyUDUgn,XbNa—?‘BelShrX—:a—sBZJs()b-t!PhTKIJeMk
UWRbBgW-so?oRb!”w—TogFItBcEXP-nE!- ,e!,j.DAT-fC“D‘k.
